In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [2]:
df=pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
df.duplicated().sum()

0

In [12]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [13]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [14]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [17]:
df=pd.get_dummies(df,columns=['Gender','Geography'],drop_first=True)

In [18]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(df.drop(columns='Exited'),df['Exited'],test_size=0.2,random_state=3)
X_train.shape,X_test.shape

((8000, 11), (2000, 11))

In [23]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [24]:
X_train_scaled

array([[ 0.0862834 , -0.6601382 , -1.0446036 , ..., -1.09499335,
        -0.58023704, -0.57542576],
       [ 0.31430986, -0.75549256, -0.35146473, ...,  0.91324755,
         1.72343359, -0.57542576],
       [-1.82084701,  0.57946842, -1.73774248, ..., -1.09499335,
        -0.58023704, -0.57542576],
       ...,
       [ 0.28321534,  1.53301197, -0.35146473, ...,  0.91324755,
         1.72343359, -0.57542576],
       [-0.51487727,  0.198051  , -1.39117304, ..., -1.09499335,
        -0.58023704, -0.57542576],
       [-0.7325389 , -0.37407514,  1.72795189, ...,  0.91324755,
         1.72343359, -0.57542576]])

In [25]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [31]:
model=Sequential()

model.add(Dense(3,activation='sigmoid',input_dim=11))
model.add(Dense(1,activation='sigmoid',input_dim=1))

In [32]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 3)                 36        
                                                                 
 dense_10 (Dense)            (None, 1)                 4         
                                                                 
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [58]:
history=model.fit(X_train_scaled,Y_train,epochs=100,validation_split=0.2)

Epoch 1/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3937 - accuracy: 0.8347 - val_loss: 0.3959 - val_accuracy: 0.8381
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3937 - accuracy: 0.8348 - val_loss: 0.3962 - val_accuracy: 0.8375
Epoch 3/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8345 - val_loss: 0.3961 - val_accuracy: 0.8369
Epoch 4/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3935 - accuracy: 0.8347 - val_loss: 0.3962 - val_accuracy: 0.8369
Epoch 5/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8345 - val_loss: 0.3961 - val_accuracy: 0.8369
Epoch 6/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8353 - val_loss: 0.3961 - val_accuracy: 0.8381
Epoch 7/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8347 - val_loss: 0.3960 - val_accuracy: 0.8388

In [51]:
model.layers[0].get_weights()

[array([[ 0.11895695, -0.01847476,  0.11008043],
        [-2.9802463 ,  0.84383327, -2.3135629 ],
        [ 0.06592745,  0.11725389, -0.1930762 ],
        [ 0.13741936, -0.02609548, -1.2302579 ],
        [ 0.12623805, -0.07070065,  0.24328987],
        [-0.14212991,  0.24976124, -0.26579198],
        [-0.5167706 ,  1.5022434 ,  0.05941416],
        [-0.07118256,  0.01682538, -0.11667316],
        [-0.01037559,  0.2086815 ,  1.1902071 ],
        [-0.26695943, -0.17963247, -0.8937306 ],
        [-0.05919769,  0.23781708, -0.60562205]], dtype=float32),
 array([ 1.3972412, -2.498504 , -0.8101596], dtype=float32)]

In [52]:
y_log=model.predict(X_test_scaled)

63/63 [==============================] - 0s 1ms/step


In [53]:
Y_pred=np.where(y_log>0.5,1,0)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.8425